In [1]:
import json
import pandas as pd
from v2_rebalance_dashboard.constants import (
    ROOT_DIR,
    eth_client,
    AUTOPOOL_ETH_STRATEGY_ABI,
    AUTOPOOL_VAULT_ABI,
    balETH_AUTOPOOL_ETH_ADDRESS,
    balETH_AUTOPOOL_ETH_STRATEGY_ADDRESS,
)
import plotly.express as px
from v2_rebalance_dashboard.get_rebalance_events_summary import fetch_clean_rebalance_events
from v2_rebalance_dashboard.solver_rebalance_plans_plots import load_balETH_solver_df, autoETH, balETH
import plotly.graph_objects as go

autoETH = "0x49C4719EaCc746b87703F964F09C22751F397BA0"
balETH = "0x72cf6d7C85FfD73F18a83989E7BA8C1c30211b73"

balETH_solver_df, balETH_proposed_rebalances_df, destination_df, rebalance_event_df = load_balETH_solver_df()

In [6]:
from v2_rebalance_dashboard.get_events import get_each_event_in_contract

vault_events  = get_each_event_in_contract(eth_client.eth.contract(balETH_AUTOPOOL_ETH_ADDRESS, abi=AUTOPOOL_VAULT_ABI), )
strategy_events = get_each_event_in_contract(eth_client.eth.contract(balETH_AUTOPOOL_ETH_STRATEGY_ADDRESS, abi=AUTOPOOL_ETH_STRATEGY_ABI), )

In [7]:
for k, df in strategy_events.items():
    print(k, df.shape)

DestTrimRebalanceDetails (0, 4)
DestViolationMaxTrimAmount (0, 4)
DustPositionPortionSet (0, 4)
IdleThresholdsSet (0, 4)
Initialized (1, 6)
LstPriceGapSet (0, 4)
PauseStart (0, 4)
PauseStop (0, 4)
RebalanceBetweenDestinations (5, 11)
RebalanceToIdle (0, 4)
RebalanceToIdleReason (0, 4)
SuccessfulRebalanceBetweenDestinations (3, 9)


In [8]:
for k, df in vault_events.items():
    print(k, df.shape)

AddedToRemovalQueue (0, 4)
Approval (0, 4)
Deposit (5, 9)
DestinationDebtReporting (0, 4)
DestinationVaultAdded (15, 6)
DestinationVaultRemoved (1, 6)
FeeCollected (0, 4)
Initialized (1, 6)
Nav (52, 8)
Paused (0, 4)
PeriodicFeeCollected (0, 4)
RemovedFromRemovalQueue (0, 4)
RewarderSet (1, 7)
Shutdown (0, 4)
SymbolAndDescSet (0, 4)
TokensPulled (0, 4)
TokensRecovered (0, 4)
Transfer (84, 8)
Unpaused (0, 4)
Withdraw (0, 4)
WithdrawalQueueSet (0, 4)


In [ ]:
def block_to_date(block:int):
    return pd.to_datetime(eth_client.eth.getBlock(block).timestamp, unit='s')

def make_time_since_last_nav_event(nav_df:pd.DataFrame):
    time_diff_hours = nav_df['date'].diff().dt.total_seconds()[2:] / 3600
    nav_df['date'] = nav_df['block'].apply(block_to_date)
    time_diff_hours.index = nav_df['date'][2:]
    fig = px.line(time_diff_hours, labels={'value': 'Hours since last Nav Event', 'index': 'Date'})
    fig.add_hline(y=24, line_dash="dash", line_color="red", annotation_text="24-hour threshold", annotation_position="top right")
    fig.update_yaxes(range=[20, 25])
    return fig